In [0]:
import nltk
import random
import math
import string
import re
import numpy as np
import collections
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm import MLE
from nltk.tokenize import sent_tokenize
from tensorflow import keras
import tensorflow as tf
from tensorflow.contrib import rnn
import time
#nltk.download('punkt')
from sklearn.model_selection import train_test_split

/home/jenil/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jenil/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jenil/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jenil/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

In [0]:
f=open("trump.txt","r")
content=f.read()

In [0]:
content=content.replace('\n','')
content=content.lower()

In [0]:
sentences=sent_tokenize(content)
sentences=['<s> '+re.sub(r'[^\w\s]','',s)+' </s>' for s in sentences]

In [0]:
random.shuffle(sentences)
train_data=sentences[:math.ceil(0.8*len(sentences))]
test_data=sentences[math.ceil(0.8*len(sentences)):]
words=[]
train_words=[]
test_words=[]
for i in sentences:
    for j in i.split():
        words.append(j)

for i in train_data:
    for j in i.split():
        train_words.append(j)

for i in test_data:
    for j in i.split():
        test_words.append(j)


In [0]:
sentence_index={}
for i in range(len(train_data)):
    sentence_index[train_data[i]]=i
    


In [0]:
logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"



def read_data(content):
    content = [x.lstrip('<s>').rstrip('</s>') for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content


def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

training_data=read_data(train_data)
testing_data=read_data(test_data)
dictionary, reverse_dictionary = build_dataset(training_data)
test_dictionary,test_reverse_dictionary=build_dataset(testing_data)
vocab_size = len(dictionary)
test_vocab_size=len(test_dictionary)



In [0]:

def Vanilla(x, weights, biases):
    #tf.reset_default_graph()
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    rnn_cell = rnn.BasicRNNCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']



In [0]:
learning_rate = 0.0001
training_iters = 10000
display_step = 1000

# number of units in RNN cell
n_hidden = 128
n_input=5
# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

pred = Vanilla(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    inc=1
    while True:
        if inc>5:
            break
        prompt = "sentence %s: " % inc
        inc=inc+1
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, len(words), 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Iter= 1000, Average Loss= 30.551778, Average Accuracy= 0.10%
['competitor', 'we', 'dont', 'have', 'strength'] - [in] vs [men]
Iter= 2000, Average Loss= 23.287459, Average Accuracy= 0.20%
['want', 'to', 'mention',

Iter= 8000, Average Loss= 8.324401, Average Accuracy= 3.10%
['states', 'that', 'weve', 'won', 'and'] - [to] vs [haveno]
Iter= 9000, Average Loss= 8.290184, Average Accuracy= 4.40%
['the', 'public', 'we', 'have', 'people'] - [that] vs [to]
Iter= 10000, Average Loss= 8.070123, Average Accuracy= 3.80%
['you', 'in', 'by', 'now', 'heavens'] - [sake] vs [the]
Optimization Finished!
Elapsed time:  1.5796981533368428 min
sentence 1: a
Word not in dictionary
sentence 2: a
Word not in dictionary
sentence 3: a
Word not in dictionary
sentence 4: a
Word not in dictionary
sentence 5: a
Word not in dictionary


In [0]:
train_perplexity = tf.exp(loss)

In [0]:
with tf.Session() as session: 
  print("perplexity for vanilla rnn",session.run(train_perplexity))

perplexity for vanilla rnn 643627.9


In [0]:
def lstm(x, weights, biases):
    #tf.reset_default_graph()
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    rnn_cell = rnn.MultiRNNCell([rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE),rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE)])

    
    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']



In [0]:
learning_rate = 0.0001
training_iters = 10000
display_step = 1000

# number of units in RNN cell
n_hidden = 128
n_input=5
# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

pred = lstm(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    inc=1
    while True:
        if inc>5:
            break
        prompt = "sentence %s: " % inc
        inc=inc+1
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, len(words), 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


Iter= 1000, Average Loss= 9.342321, Average Accuracy= 1.00%
['in', 'our', 'country', 'in', 'other'] - [words] vs [you]
Iter= 2000, Average Loss= 8.347518, Average Accuracy= 3.70%
['one', 'of', 'the', 'guys', 'id'] - [definitely] vs [i]
Iter= 3000, Average Loss= 7.807893, Average Accuracy= 2.60%
['watch', 'that', 'longhillarys', 'debate', 'essentially'] - [lasted] vs [and]
Iter= 4000, Average Loss= 7.456790, Average Accuracy= 4.00%
['be', 'knocked', 'to', 'hell', 'you'] - [could] vs [and]


Iter= 5000, Average Loss= 7.293267, Average Accuracy= 5.10%
['although', 'i', 'guess', 'we', 'just'] - [did] vs [i]
Iter= 6000, Average Loss= 7.147050, Average Accuracy= 5.30%
['only', 'thing', 'that', 'stops', 'the'] - [crowds] vs [the]
Iter= 7000, Average Loss= 6.975493, Average Accuracy= 3.70%
['i', 'build', 'great', 'buildings', 'how'] - [we] vs [to]
Iter= 8000, Average Loss= 6.991382, Average Accuracy= 4.00%
['to', 'win', 'by', 'the', 'margins'] - [and] vs [i]
Iter= 9000, Average Loss= 6.993953, Average Accuracy= 4.80%
['that', 'are', 'criminals', 'we', 'have'] - [people] vs [to]
Iter= 10000, Average Loss= 7.068118, Average Accuracy= 4.00%
['sake', 'you', 'might', 'have', 'died'] - [now] vs [the]
Optimization Finished!
Elapsed time:  3.873773733774821 min


In [0]:
train_perplexity = tf.exp(loss)

In [0]:
with tf.Session() as session: 
  print("perplexity for lstm ",session.run(train_perplexity))

21199.5


The neural network performs better than classical approaches. Neural network has shot off in many fields like image processing, natural language processing, pattern recognition, etc.
1. Neural performs better than the classical approach when there is a large amount of data. The amount of data has grown due to digitalization. These data help improve learning.
2. When the domain knowledge is a little less, the neural networks are better than classic as we do not have to find hand-engineered features. The model itself finds the features.